<a href="https://colab.research.google.com/github/nandita2291/Medusa/blob/main/final_website_waste_segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEP LEARNING MODEL FOR MEDICAL WASTE SEGREGATION (STAGE 1 and 2)

This code integrates the two stages of waste segregation. The first stage segregates the waste into chemical, sharp and infectious. The second stage classifies based on method of segregation. The classes we have used are incinerable, non-biodegradable and recyclable. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from keras import Sequential
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/stage2-wastesegregation.zip'

Archive:  /content/drive/MyDrive/stage2-wastesegregation.zip
   creating: stage2-wastesegregation/train/
   creating: stage2-wastesegregation/train/incinerable/
  inflating: stage2-wastesegregation/train/incinerable/1 (10).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (100).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (101).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (102).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (103).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (104).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (105).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (106).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (107).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (108).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (109).jpeg  
  inflating: stage2-wastesegregation/train/incinerable/1 (11).jpeg  
 

In [4]:
train_dir= image.ImageDataGenerator(height_shift_range=0.1,width_shift_range=0.1,rotation_range=25,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_dir= image.ImageDataGenerator(height_shift_range=0.1,width_shift_range=0.1,rotation_range=25,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [5]:
train1_set= train_dir.flow_from_directory(
    '/content/stage2-wastesegregation/train',
    batch_size=8,
    class_mode='categorical',
    target_size=(224,224)
)
test1_set= test_dir.flow_from_directory(
    '/content/stage2-wastesegregation/validation',
    shuffle=False,
    batch_size=8,
    class_mode='categorical',
    target_size=(224,224)
)

Found 849 images belonging to 3 classes.
Found 184 images belonging to 3 classes.


In [6]:
train2_set= train_dir.flow_from_directory(
    '/content/drive/MyDrive/waste_segregation/train',
    batch_size=8,
    class_mode='categorical',
    target_size=(224,224)
)
test2_set= test_dir.flow_from_directory(
    '/content/drive/MyDrive/waste_segregation/validation',
    shuffle=False,
    batch_size=8,
    class_mode='categorical',
    target_size=(224,224)
)

Found 1054 images belonging to 3 classes.
Found 216 images belonging to 3 classes.


In [7]:
base1_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base1_model.layers:
    layer.trainable=False
model1 = Sequential()
model1.add(base1_model)
model1.add(GaussianNoise(0.25))
model1.add(GlobalAveragePooling2D())
model1.add(Dense(750,activation='relu'))
model1.add(BatchNormalization()) 
model1.add(GaussianNoise(0.25))
model1.add(Dropout(0.25))
model1.add(Dense(3, activation='softmax'))
model1.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 gaussian_noise (GaussianNoi  (None, 7, 7, 1280)       0         
 se)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 750)               960750    
                                                                 
 batch_normalization (BatchN  (None, 750)              3000

In [8]:
base2_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base2_model.layers:
    layer.trainable=False
model2 = Sequential()
model2.add(base1_model)
model2.add(GaussianNoise(0.25))
model2.add(GlobalAveragePooling2D())
model2.add(Dense(750,activation='relu'))
model2.add(BatchNormalization()) 
model2.add(GaussianNoise(0.25))
model2.add(Dropout(0.25))
model2.add(Dense(3, activation='softmax'))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 gaussian_noise_2 (GaussianN  (None, 7, 7, 1280)       0         
 oise)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 750)               960750    
                                                                 
 batch_normalization_1 (Batc  (None, 750)              3000      
 hNormalization)                                      

In [9]:
model1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy','Precision','Recall','AUC'])

In [10]:
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy','Precision','Recall','AUC'])

In [11]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_loss',patience=5)
history=model1.fit(
    train1_set,
    epochs=15,
    steps_per_epoch= 90,
    validation_data= test1_set
)

Epoch 1/15


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


90/90 [==============================] - 32s 181ms/step - loss: 1.1797 - accuracy: 0.6017 - precision: 0.6201 - recall: 0.5792 - auc: 0.7792 - val_loss: 1.4805 - val_accuracy: 0.5870 - val_precision: 0.5886 - val_recall: 0.5598 - val_auc: 0.7234
Epoch 2/15
90/90 [==============================] - 13s 144ms/step - loss: 0.7346 - accuracy: 0.7194 - precision: 0.7411 - recall: 0.6958 - auc: 0.8763 - val_loss: 0.9583 - val_accuracy: 0.6793 - val_precision: 0.6988 - val_recall: 0.6304 - val_auc: 0.8229
Epoch 3/15
90/90 [==============================] - 13s 147ms/step - loss: 0.6498 - accuracy: 0.7560 - precision: 0.7732 - recall: 0.7363 - auc: 0.8957 - val_loss: 1.9394 - val_accuracy: 0.5000 - val_precision: 0.5057 - val_recall: 0.4837 - val_auc: 0.6939
Epoch 4/15
90/90 [==============================] - 13s 150ms/step - loss: 0.5874 - accuracy: 0.7686 - precision: 0.7763 - recall: 0.7447 - auc: 0.9118 - val_loss: 1.1243 - val_accuracy: 0.6359 - val_precision: 0.6474 - val_recall: 0.6087 -

In [13]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor='val_loss',patience=5)
history1=model2.fit(
    train2_set,
    epochs=15,
    steps_per_epoch= 90,
    validation_data= test2_set
)

Epoch 1/15
90/90 [==============================] - 134s 1s/step - loss: 0.8235 - accuracy: 0.7069 - precision: 0.7271 - recall: 0.6847 - auc: 0.8575 - val_loss: 0.8360 - val_accuracy: 0.7083 - val_precision: 0.7225 - val_recall: 0.6991 - val_auc: 0.8538
Epoch 2/15
45/90 [==============>...............] - ETA: 23s - loss: 0.7340 - accuracy: 0.7278 - precision: 0.7328 - recall: 0.7083 - auc: 0.8787

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


90/90 [==============================] - 56s 627ms/step - loss: 0.7127 - accuracy: 0.7256 - precision: 0.7448 - recall: 0.7033 - auc: 0.8834 - val_loss: 0.7794 - val_accuracy: 0.7269 - val_precision: 0.7600 - val_recall: 0.7037 - val_auc: 0.8652
Epoch 3/15
90/90 [==============================] - 28s 311ms/step - loss: 0.6323 - accuracy: 0.7479 - precision: 0.7668 - recall: 0.7326 - auc: 0.9008 - val_loss: 0.7496 - val_accuracy: 0.7546 - val_precision: 0.7783 - val_recall: 0.7315 - val_auc: 0.8823
Epoch 4/15
90/90 [==============================] - 22s 245ms/step - loss: 0.5769 - accuracy: 0.7702 - precision: 0.7976 - recall: 0.7521 - auc: 0.9179 - val_loss: 0.8950 - val_accuracy: 0.6991 - val_precision: 0.7184 - val_recall: 0.6852 - val_auc: 0.8560
Epoch 5/15
90/90 [==============================] - 18s 200ms/step - loss: 0.5825 - accuracy: 0.7855 - precision: 0.8029 - recall: 0.7660 - auc: 0.9159 - val_loss: 0.7952 - val_accuracy: 0.7269 - val_precision: 0.7286 - val_recall: 0.7083 -

In [14]:
model1.evaluate(train1_set)

 71/107 [==================>...........] - ETA: 4s - loss: 0.4580 - accuracy: 0.8169 - precision: 0.8333 - recall: 0.8011 - auc: 0.9455

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


107/107 [==============================] - 12s 114ms/step - loss: 0.4696 - accuracy: 0.8068 - precision: 0.8223 - recall: 0.7903 - auc: 0.9422


[0.4696170389652252,
 0.8068315386772156,
 0.8223039507865906,
 0.7903415560722351,
 0.9422382116317749]

In [15]:
model2.evaluate(train2_set)

 27/132 [=====>........................] - ETA: 13s - loss: 0.4198 - accuracy: 0.8333 - precision: 0.8578 - recall: 0.8102 - auc: 0.9538

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


132/132 [==============================] - 18s 134ms/step - loss: 0.3694 - accuracy: 0.8653 - precision: 0.8917 - recall: 0.8435 - auc: 0.9649


[0.3694298565387726,
 0.8652751445770264,
 0.8916749954223633,
 0.8434535264968872,
 0.964870810508728]

In [16]:
model1.evaluate(test1_set)

23/23 [==============================] - 3s 133ms/step - loss: 0.9196 - accuracy: 0.6848 - precision: 0.6780 - recall: 0.6522 - auc: 0.8419


[0.9195517897605896,
 0.6847826242446899,
 0.6779661178588867,
 0.6521739363670349,
 0.8419262170791626]

In [17]:
model2.evaluate(test2_set)

27/27 [==============================] - 4s 154ms/step - loss: 0.7559 - accuracy: 0.7407 - precision: 0.7778 - recall: 0.7130 - auc: 0.8756


[0.7558643817901611,
 0.7407407164573669,
 0.7777777910232544,
 0.7129629850387573,
 0.8756376504898071]

In [18]:
model1.save('stage2-waste_segregator.h5')

In [19]:
model2.save('stage1-waste_segregator.h5')

In [20]:
!pip install streamlit

     |████████████████████████████████| 10.1 MB 4.3 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 4.3 MB 34.1 MB/s 
     |████████████████████████████████| 111 kB 57.3 MB/s 
     |████████████████████████████████| 164 kB 44.0 MB/s 
     |████████████████████████████████| 181 kB 44.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 130 kB 52.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 793 kB 39.5 MB/s 
     |████████████████████████████████| 130 kB 54.9 MB/s 
     |████████████████████████████████| 428 kB 53.7 MB/s 
     |████████████████████████████████| 380 kB 52.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=b06a7832b96f12a20f378e60552606815725ccc6c96a4566ecd2d8ee60270baa
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf

The website has three different sections. The first section is about the introduction and the different types of medical wastes the tool deals with. The second section is the deep learning tool where the user can send an image and find into which category the waste belongs to. The last section is about the team.

In [23]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import streamlit as st
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageOps
import pandas as pd
col1, col2, col3 = st.columns([1,6,1])

with col1:
   st.write("                       ")

with col2:
   logo = Image.open('logo.png')
   st.image(logo, width=250)

with col3:
   st.write("")

html_temp = """ 
  <div style="background-color:pink ;padding:10px"> 
  <h2 style="color:white;text-align:center;">MEDUSA- MEDical Utility Segregation Algorithm ⚕️⚕️⚕️</h2>
  </div>
  """ 
st.markdown(html_temp, unsafe_allow_html=True) 
st.subheader('DONE BY')
st.text('1. SAIRAM.V.A')
st.text('2. CHRISTAN RAYAN')
st.text('3. NANDITA SAJEEV')
st.text('4. SAMYUKTHA KAPOOR')
activities=['SECTION 1-Introduction','SECTION 2- Stage 1 Deep learning tool for medical waste segregation','SECTION 3- Stage 2 Deep learning tool for medical waste segregation','SECTION 4- About Us']
option=st.sidebar.selectbox('choose the options displayed below',activities)
if option=='SECTION 1-Introduction':
  st.write('this website is a deep learning application for improvised and efficient waste segregation. In this the user will upload the image of the waste and the model will predict the category of the waste along with the color code of the bin.')
  st.write('Segregation refers to the basic separation of different categories of waste generated at source and thereby reducing the risks as well as cost of handling and disposal. Segregation is the most crucial step in bio-medical waste management. Effective segregation alone can ensure effective biomedical waste management.')
  st.subheader('CHEMICAL WASTE:')
  image = Image.open('im3.jpg')
  st.image(image) 
  st.write('Medical batteries, broken thermometers and blood pressure gauges that contain mercury, radiological liquids, chemical reagents, electrode gel are some examples of chemical waste')
  st.subheader('INFECTIOUS WASTE:')
  imag = Image.open('im2.jpg')
  st.image(imag)
  st.write('Cotton balls, tissues, ear buds, swabs, sanitary napkins, gloves, masks that are in contact with body fluids are some examples of infectious wastes.')
  st.subheader('SHARP WASTE')
  ima = Image.open('im1.jpg')
  st.image(ima)
  st.write('Cut nails, Tweezerzs, surgical instruments, pins, syringe, razors, broken glass (except thermometer and bp gauge), guide wires, lancet, pricking device are certain examples.')
  st.subheader('RECYCLABLE WASTE')
  st.write('Recycling is the process of converting waste materials into new materials and objects. The recovery of energy from waste materials is often included in this concept.Recyclable materials include many kinds of glass, paper, cardboard, metal, plastic, tires, textiles, batteries, and electronics. The composting and other reuse of biodegradable waste such as food and garden waste is also a form of recycling.')
  st.subheader('NON-BIODEGRADABLE WASTE')
  st.write('Non-biodegradable waste is defined as a substance that cannot be decomposed or dissolved naturally and acts as a source of pollution. They may remain on this planet for many years without any degradation, thereby posing a critical threat to the environment. Non-biodegradable waste examples include- plastics, metal, aluminium cans, tyres, pains, toxic chemicals, toxic chemicals, polystyrene.')
  st.subheader('INCINERABLE WASTE')
  st.write('Incinerable waste are wastes that can be burnt in an incinerator. These are any wastes which are combustible and contain a calorific value of at least 2000 kcal/kg of material.Only those materials with very low calorific value and with recyclable properties can be incinerated. Three types of waste to which incineration is applied extensively are municipal solid waste, hazardous waste, and medical waste.')
elif option=='SECTION 2- Stage 1 Deep learning tool for medical waste segregation':
  html_temp = """ 
  <div style="background-color:orange ;padding:10px">
  <h2 style="color:black;text-align:center;"> Stage 1 Deep learning tool for medical waste segregation</h2>
  </div>
  """ 
  st.markdown(html_temp, unsafe_allow_html=True) 
  @st.cache(allow_output_mutation=True)
  def load_model():
    model=tf.keras.models.load_model('/content/stage1-waste_segregator.h5') #load the trained model
    return model
  with st.spinner('Model is being loaded..'): #just a delay message pops up to kill the time for loading
    model=load_model()
  file = st.file_uploader("Please upload any image from the local machine in case of computer or upload camera image in case of mobile.", type=["jpg", "png","jpeg"])
  import cv2
  import numpy as np
  st.set_option('deprecation.showfileUploaderEncoding', False)
  def import_and_predict(image_data, model):
    
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS) #resize the data without aliasing them.
        image = np.asarray(image) #convert to array to feed it as output for model
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_reshape = img[np.newaxis,...]
        prediction = model.predict(img_reshape)
        return prediction
  if file is None:
     st.text("Please upload an image file within the allotted file size")
  else:
     img = Image.open(file)
     st.image(img, use_column_width=False)
     size = (224,224)    
     image = ImageOps.fit(img, size, Image.ANTIALIAS)
     imag = np.asarray(image)
     imaga = np.expand_dims(imag,axis=0) 
     predictions = model.predict(imaga) #it returns an 1-D array
     a= np.argmax(predictions,-1) #this finds out the class to which the waste image belongs to
     input= st.radio('Enter the suitable language',('Tamil','Telugu','Hindi','English'))
     if a==[0] and input=='English':
       st.success('The given waste belongs to CHEMICAL category. Please ensure that you dispose it in the YELLOW color bin.')
       yellow = Image.open('yellow.jpg')
       st.image(yellow)
     elif a==[0] and input=='Tamil':  
       st.success('கொடுக்கப்பட்ட கழிவுகள் இரசாயன வகையைச் சேர்ந்தது. அதை மஞ்சள் கலர் தொட்டியில் அப்புறப்படுத்துவதை உறுதி செய்யவும்.')
       yellow = Image.open('yellow.jpg')
       st.image(yellow)
     elif a==[0] and input=='Telugu':
       st.success('ఇచ్చిన వ్యర్థాలు రసాయన వర్గానికి చెందినవి. దయచేసి మీరు దానిని పసుపు రంగు బిన్‌లో పారవేసినట్లు నిర్ధారించుకోండి.')
       yellow = Image.open('yellow.jpg')
       st.image(yellow) 
     elif a==[0] and input=='Hindi':
       st.success('दिया गया कचरा केमिकल कैटेगरी का है। कृपया सुनिश्चित करें कि आप इसे पीले रंग के बिन में फेंक दें।')
       yellow = Image.open('yellow.jpg')
       st.image(yellow)    
     elif a==[1] and input=='English':
       st.success('The given waste belongs to INFECTIOUS category. Please ensure that you dispose it in the RED color bin')
       red = Image.open('red.jpg')
       st.image(red)
     elif a==[1] and input=='Tamil':
       st.success('கொடுக்கப்பட்ட கழிவுகள் தொற்று வகையைச் சேர்ந்தது. சிவப்பு வண்ணத் தொட்டியில் அதை அப்புறப்படுத்துவதை உறுதிசெய்யவும்.')
       red = Image.open('red.jpg')
       st.image(red)
     elif a==[1] and input=='Telugu':
       st.success('ఇచ్చిన వ్యర్థాలు అంటువ్యాధి వర్గానికి చెందినవి. దయచేసి మీరు దానిని RED కలర్ బిన్‌లో పారవేసినట్లు నిర్ధారించుకోండి.')
       red = Image.open('red.jpg')
       st.image(red)
     elif a==[1] and input=='Hindi':
       st.success('दिया गया कचरा संक्रामक श्रेणी का है। कृपया सुनिश्चित करें कि आप इसे लाल रंग के बिन में फेंक दें।')
       red = Image.open('red.jpg')
       st.image(red)    
     elif a==[2] and input=='English':
       st.success('The given waste belongs to SHARP category. Please ensure that you dispose it in the WHITE color bin')
       white = Image.open('white.jpg')
       st.image(white)
     elif a==[2] and input=='Tamil':
       st.success('கொடுக்கப்பட்ட கழிவுகள் SHARP வகையைச் சேர்ந்தது. வெள்ளை வண்ணத் தொட்டியில் அதை அப்புறப்படுத்துவதை உறுதிசெய்யவும்.')
       white = Image.open('white.jpg')
       st.image(white) 
     elif a==[2] and input=='Telugu':
       st.success('கொடுக்கப்பட்ட கழிவுகள் SHARP வகையைச் சேர்ந்தது. வெள்ளை வண்ணத் தொட்டியில் அதை அப்புறப்படுத்துவதை உறுதிசெய்யவும்.')
       white = Image.open('white.jpg')
       st.image(white)
     elif a==[2] and input=='Hindi':
       st.success('दिया गया कचरा शार्प कैटेगरी का है। कृपया सुनिश्चित करें कि आप इसे सफेद रंग के बिन में फेंक दें।')
       white = Image.open('white.jpg')
       st.image(white)   
     st.warning('the model is only 85% accurate. Futher enhancements have to made to get the best results.')
elif option=='SECTION 3- Stage 2 Deep learning tool for medical waste segregation':
  html_temp = """ 
  <div style="background-color:pink ;padding:10px">
  <h2 style="color:black;text-align:center;"> Stage 2 Deep learning tool for medical waste segregation</h2>
  </div>
  """ 
  st.markdown(html_temp, unsafe_allow_html=True) 
  @st.cache(allow_output_mutation=True)
  def load_model():
    model=tf.keras.models.load_model('/content/stage2-waste_segregator.h5') #load the trained model
    return model
  with st.spinner('Model is being loaded..'): #just a delay message pops up to kill the time for loading
    model=load_model()
  file = st.file_uploader("Please upload any image from the local machine in case of computer or upload camera image in case of mobile.", type=["jpg", "png","jpeg"])
  import cv2
  import numpy as np
  st.set_option('deprecation.showfileUploaderEncoding', False)
  def import_and_predict(image_data, model):
    
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS) #resize the data without aliasing them.
        image = np.asarray(image) #convert to array to feed it as output for model
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_reshape = img[np.newaxis,...]
        prediction = model.predict(img_reshape)
        return prediction
  if file is None:
     st.text("Please upload an image file within the allotted file size")
  else:
     img = Image.open(file)
     st.image(img, use_column_width=False)
     size = (224,224)    
     image = ImageOps.fit(img, size, Image.ANTIALIAS)
     imag = np.asarray(image)
     imaga = np.expand_dims(imag,axis=0) 
     predictions = model.predict(imaga) #it returns an 1-D array
     a= np.argmax(predictions,-1) #this finds out the class to which the waste image belongs to
     inp=st.radio('Enter the suitable language',('English','Tamil','Telugu','Hindi'))
     if a==[0] and inp=='English':
       st.success('The given waste belongs to INCINERABLE category. Please ensure that you dispose it in the YELLOW color bin.')
       yellow = Image.open('yellow.jpg')
       st.image(yellow)
     elif a==[0] and inp=='English':
       st.success('The given waste belongs to INCINERABLE category. Please ensure that you dispose it in the YELLOW color bin.')
       yellow = Image.open('yellow.jpg')
       st.image(yellow)  
     elif a==[1]:
       st.success('The given waste belongs to NON-BIO-DEGRADABLE category. Please ensure that you dispose it in the RED color bin')
       red = Image.open('red.jpg')
       st.image(red)
     else:
       st.success('The given waste belongs to RECYCLABLE category. Please ensure that you dispose it in the WHITE color bin')
       white = Image.open('white.jpg')
       st.image(white)
     st.warning('the model is only 85% accurate. Futher enhancements have to made to get the best results.')
elif option=='SECTION 4- About Us':
  st.subheader('1. V.A.SAIRAM')
  st.success('Third year Biomedical Engineering, Rajalakshmi Engineering College')
  st.subheader('2. CHRISTAN RAYAN')
  st.success('Third year Mechatronics Engineering, Rajalakshmi Engineering College')
  st.subheader('3. NANDITHA SAJEEV')
  st.success('Third year Computer Science Engineering, Rajalakshmi Engineering College')
  st.subheader('4. SAMYUKTHA KAPOOR')
  st.success('Third year Biomedical Engineering, Rajalakshmi Engineering College')


Overwriting app.py


In [24]:
!streamlit run app.py & npx localtunnel --port 8501

2022-04-02 06:43:19.547 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 3.122s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.80.51.165:8501

your url is: https://new-vampirebat-15.loca.lt
2022-04-02 06:45:11.160607: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
  Stopping...
^C
